In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
sys.path.insert(0, '../scripts')
from utils import *

In [2]:
plt.style.use('seaborn')

In [3]:
plt.rcParams['figure.figsize'] = (12, 10)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using MXNet backend.


In [6]:
import datetime
import time

In [7]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Merge, Flatten, Dense, Reshape, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.engine.topology import merge

In [8]:
train_data = pd.read_csv('../data/clean-train.csv').dropna(axis=0)
test_data = pd.read_csv('../data/clean-test.csv').dropna(axis=0)

In [9]:
q1_list = train_data['q1'].tolist()
q2_list = train_data['q2'].tolist()

In [10]:
vocab_size = 10000

In [11]:
token = Tokenizer(nb_words=vocab_size)
token.fit_on_texts(q1_list + q2_list)

In [12]:
question1_seq = token.texts_to_sequences(q1_list)
question2_seq = token.texts_to_sequences(q2_list)

In [13]:
MAX_SEQUENCE_LENGTH = 25

In [14]:
q1_data = pad_sequences(question1_seq, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(question2_seq, maxlen=MAX_SEQUENCE_LENGTH)

In [15]:
train_labels = train_data['is_same'].values

In [16]:
MAX_SEQUENCE_LENGTH = 25
WORD_EMBEDDING_DIM = 300
SENT_EMBEDDING_DIM = 128
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.1
RNG_SEED = 13371447
NB_EPOCHS = 150
DROPOUT = 0.3
dense_size = 300
BATCH_SIZE = 516
nb_words = vocab_size

In [17]:
gpus = 8
gpu_list = []
for i in range(gpus):
    gpu_list.append('gpu({:.0f})'.format(i))

In [26]:
### Adapted from https://github.com/bradleypallen/keras-quora-question-pairs

question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(nb_words + 1, 
                 WORD_EMBEDDING_DIM, 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=True)(question1)
q1 = Bidirectional(LSTM(SENT_EMBEDDING_DIM, return_sequences=True), merge_mode="sum")(q1)

q2 = Embedding(nb_words + 1, 
                 WORD_EMBEDDING_DIM, 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=True)(question2)
q2 = Bidirectional(LSTM(SENT_EMBEDDING_DIM, return_sequences=True), merge_mode="sum")(q2)

attention = merge([q1,q2], mode='dot', dot_axes=1)
attention = Flatten()(attention)
attention = Dense((MAX_SEQUENCE_LENGTH*SENT_EMBEDDING_DIM))(attention)
attention = Reshape((MAX_SEQUENCE_LENGTH, SENT_EMBEDDING_DIM))(attention)

merged = merge([q1,attention], mode='sum', concat_axis=1)
merged = Flatten()(merged)
merged = Dense(dense_size, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(dense_size, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(dense_size, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(dense_size, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)

In [27]:
model = Model([question1,question2], is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'], context=gpu_list)

In [28]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint('../Saved Models/lstm-attention', monitor='val_acc', save_best_only=True),
            EarlyStopping(monitor='val_acc', patience=0)]
history = model.fit([q1_data, q2_data],
                    train_labels,
                    nb_epoch=NB_EPOCHS,
                    validation_split=VALIDATION_SPLIT,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Starting training at 2017-12-17 16:48:36.713332
Train on 291067 samples, validate on 32341 samples
Epoch 1/150


/home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/module/bucketing_module.py:385: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.001937984496124031). Is this intended?
  force_init=force_init)


291067/291067 [==============================] - 173s - loss: 0.5750 - acc: 0.7033 - val_loss: 0.6879 - val_acc: 0.6297
Epoch 2/150
291067/291067 [==============================] - 163s - loss: 0.4875 - acc: 0.7634 - val_loss: 1.1904 - val_acc: 0.6606
Epoch 3/150
291067/291067 [==============================] - 163s - loss: 0.4491 - acc: 0.7870 - val_loss: 0.5564 - val_acc: 0.7330
Epoch 4/150
291067/291067 [==============================] - 161s - loss: 0.4158 - acc: 0.8071 - val_loss: 0.5237 - val_acc: 0.7233
Training ended at 2017-12-17 16:59:39.267344
Minutes elapsed: 11.042561


In [21]:
q1_list_test = test_data['q1'].tolist()
q2_list_test = test_data['q2'].tolist()

question1_seq_test = token.texts_to_sequences(q1_list_test)
question2_seq_test = token.texts_to_sequences(q2_list_test)

q1_data_test = pad_sequences(question1_seq_test, maxlen=MAX_SEQUENCE_LENGTH)
q2_data_test = pad_sequences(question2_seq_test, maxlen=MAX_SEQUENCE_LENGTH)

test_labels = test_data['is_same'].values

In [22]:
preds = model.predict([q1_data_test, q2_data_test]).flatten()

In [23]:
classes = (preds > 0.5).astype('int').flatten()

In [24]:
generate_report(test_labels, classes.flatten(), preds.flatten())

Loss Report
-----------

Accuracy Score: 0.7283
ROC  AUC Score: 0.8168
Log Loss Score: 0.5645


(0.72830039330150642, 0.81681043224052585, 0.56454150266646941)

In [29]:
model.count_params()

60547493